This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [42]:
# First, import the relevant modules
import requests
import json
from collections import OrderedDict

In [19]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url= 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31api_key='
r = requests.get(url)
json_data = r.json()

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data['dataset_data'].keys())
print(json_data['dataset_data']['column_names'])
print(json_data['dataset_data']['data'][0])
print(type(json_data))

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]
<class 'dict'>


The json data is a nested dictionary. The inner dictionary 'dataset_data' seems to have the important data and column labels. The important 'data' is a list of lists. I need to convert this into a nested dictionary and the inner dictionary needs to be zipped together with the column names. The outer dictionary will be sorted using OrderedDict with dates as keys. 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [55]:
column_names = json_data['dataset_data']['column_names']
exchange_data = json_data['dataset_data']['data']
print(type(exchange_data))
exchange_inner= {}
exchange_ordered = OrderedDict()
for data in exchange_data: # create  a nested dictionary ordered by date
    inner = dict(zip(column_names, data))
    exchange_ordered[inner['Date']] = inner
print(exchange_ordered['2017-12-29'])

2017-12-29
<class 'list'>
{'Date': '2017-12-29', 'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0, 'Turnover': 1792304.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}


In [52]:
# Calculate what the highest and lowest opening prices were for the stock in this period.
highest_opening= []
for data in exchange_ordered:
    highest_opening.append(exchange_ordered[data]['Open'])
s_highest_opening = []
for thing in highest_opening:
    if thing is None:
        continue
    else:
        s_highest_opening.append(thing)
sorted_opening = sorted(s_highest_opening)
print(sorted_opening[0], sorted_opening[-1])

34.0 53.11


In [23]:
# What was the largest change in any one day (based on High and Low price)?
highest_price= []
for data in exchange_ordered:
    highest_price.append(exchange_ordered[data]['High'])
lowest_price= []
for data in exchange_ordered:
    lowest_price.append(exchange_ordered[data]['Low'])

In [26]:
differences = []
for index in range (len(highest_price)):
    high_low_diff = highest_price[index] - lowest_price[index]
    differences.append(high_low_diff)
print(sorted(differences)[-1])

2.8100000000000023


In [51]:
# What was the largest change between any two days (based on Closing Price)?
close = []
for data in exchange_ordered:
    close.append(exchange_ordered[data]['Close'])
close_diff = []   
for index in range (len(close[:-1])):
    daily_differences = abs(close[index + 1] - close[index])
    close_diff.append(daily_differences)
print(sorted(close_diff)[-1])

2.559999999999995


In [50]:
# What was the average daily trading volume during this year?
# (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
trade_volume= []
for data in exchange_ordered:
    trade_volume.append(exchange_ordered[data]['Traded Volume'])
print(sum(trade_volume)/len(trade_volume))
def median(lst): # from stackoverflow
    sortedLst = sorted(lst)
    lstLen = len(lst)
    index = (lstLen - 1) // 2
   
    if (lstLen % 2):
        return sortedLst[index]
    else:
        return (sortedLst[index] + sortedLst[index + 1])/2.0
print(median(trade_volume))

89124.33725490196
76286.0
